In [1]:
import os
import shutil

import geopandas as gpd
import pandas as pd
import nivapy3 as nivapy
from shapely.geometry import box

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


## Get main catchment

Also display the bounds as lat/lon.

In [3]:
# Get main catchment
vassom_gdf = nivapy.da.read_postgis(
        'physical',
        'norway_nve_vassdragomrade_poly',
        eng,
)
vassom_gdf = vassom_gdf.query("vassdragsomradenr == '234'")
vassom_gdf.bounds

,minx,miny,maxx,maxy
215,23.701473,68.552719,29.10171,70.750101


The best projected co-ordinate system to use between 24 and 30 degrees E is UTM Zone 35N (see [here](https://epsg.io/32635)). However, most of the datasets I have are national scale and use EPSG 25833, which should be good enough in most cases. Reprojecting to Zone 35N may be wise for any critical area calculations, but I won't worry about this for now.

In [4]:
# Reproject and save
vassom_gdf = vassom_gdf.to_crs(epsg=25833)
fpath = "/home/jovyan/shared/QUANTOM/data/gis/tana_vassdragsomrade.geojson"
vassom_gdf.to_file(fpath, driver="GeoJSON")

## Save bounding box
#bnds = vassom_gdf.bounds
#b = [box(l, b, r, t) for l, b, r, t in zip(bnds.minx, bnds.miny, bnds.maxx, bnds.maxy)]
#vassom_bb = gpd.GeoDataFrame(bnds, geometry=b)
#fpath = "../data/shapefiles/tana_vassdragsomrade_bounding_box.geojson"
#vassom_bb.to_file(fpath, driver="GeoJSON")

## Extract vector and raster layers of interest

Available vector layers are [here](https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md).

In [5]:
# Path to boundng box
bbox_path = "/home/jovyan/shared/QUANTOM/data/gis/tana_vassdragsomrade.geojson"

# Output folder
out_fold = "/home/jovyan/shared/QUANTOM/data/gis"

# Vector layers of interest
# See https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md
vec_layers = [
    "physical.norway_nibio_corine_2000_poly",
    "physical.norway_nibio_corine_2006_poly",
    "physical.norway_nibio_corine_2012_poly",
    "physical.norway_nibio_corine_2018_poly",
    "physical.norway_nibio_ar50_poly",
    "physical.norway_nve_elvis_river_network_line",
    "physical.norway_nve_innsjo_poly",
]

# Raster layers of interest
# See shared/01_datasets/spatial
ras_layers = [
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif",
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif",
]

In [6]:
# Read clip features
bb_gdf = gpd.read_file(bbox_path)

# Loop over layers
for layer in vec_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    schema, table = layer.split(".")
    gdf = nivapy.da.read_postgis(
        schema,
        table,
        eng,
        clip=bb_gdf,
    )

    # Reproject
    gdf = gdf.to_crs(epsg=25833)
    
    # Save
    fpath = os.path.join(out_fold, f"{table}.geojson")
    gdf.to_file(fpath, driver="GeoJSON")

Processing: physical.norway_nibio_corine_2000_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2006_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2012_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2018_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_ar50_poly
Processing: physical.norway_nve_elvis_river_network_line
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nve_innsjo_poly


In [7]:
# Loop over layers
for layer in ras_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    fname = os.path.split(layer)[1]
    out_gtiff = os.path.join(out_fold, fname)
    nivapy.spatial.clip_raster_to_gdf(
        layer,
        out_gtiff,
        bb_gdf,
        bounding_box=True,
    )

Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})
Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})


## Extract "regine" catchments

Rather than "clipping" these as with the other vector layers above, it makes more sense to just query all the regine units associated with Tana (vassdragsområde 234).

In [8]:
# Get all regine catchments within Tana
reg_gdf = nivapy.da.read_postgis(
        'physical',
        'norway_nve_regine_catchments_poly',
        eng,
)
reg_gdf = reg_gdf[reg_gdf['vassdragsnummer'].str.startswith('234')]

# Save
fpath = "/home/jovyan/shared/QUANTOM/data/gis/tana_regine.geojson"
reg_gdf.to_file(fpath, driver="GeoJSON")